# Patchy particle STMV potential 

Implempentation of the Satelitte Tobacco Mosaic virus potential based on a coarse-grained partchy particle model.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import msmrd2
import msmrd2.tools.quaternions as quats
import msmrd2.visualization as msmrdvis
from msmrd2.potentials import patchyParticleSTMV
from msmrd2.integrators import overdampedLangevin as odLangevin

In [ ]:
# Define arbitrarily list of patchy particles
numparticles = 4
boxsize = 10
D = 0.3
Drot = 0.3
pyPartlist = [] 
positionList = [np.array([-4.0,0,0]), np.array([4.,0.,0.]), np.array([0. ,4.,0.]), np.array([0.,-4.,0.])]
orientation = np.array([1.,0.,0.,0.])

for i in range(numparticles):
    part = msmrd2.particle(D, Drot, positionList[i], orientation)
    pyPartlist.append(part)

In [ ]:
# Create list of particles that can be read from msmrd
# note the particles in this list will be independent from the python list.
partlist = msmrd2.integrators.particleList(pyPartlist)

In [ ]:
# Over-damped Langevin integrator definition
dt = 0.00001 #0.000005
seed = -1
bodytype = 'rigidbody' # three rotational degrees of freedom
integrator = odLangevin(dt, seed, bodytype) 
integrator.setKbT(0.1)

In [ ]:
# Define boundary (choose either spherical or box)
boxBoundary = msmrd2.box(boxsize,boxsize,boxsize, 'reflective')
integrator.setBoundary(boxBoundary)

In [ ]:
# Define Patchy Particle potential
strength = 100.0
angularStrength = 100.0
patchySTMV = patchyParticleSTMV(strength, angularStrength)
integrator.setPairPotential(patchySTMV)

In [ ]:
# Integrate the particles, save to .xyz to produce VMD output (additional overhead)
timesteps = 5000000
stride = 250 #1000
datafile  = open('../../data/vmd/patchyParticleSTMV.xyz', 'w')
for i in range(timesteps):
    if i%stride == 0:
        datafile.write(str(8*len(partlist)) + '\n')
        datafile.write(str(0) + '\n')
    for j, part in enumerate(partlist):
        if i%stride == 0:
            m1 = patchySTMV.getPartPosition('m1', part)
            m2 = patchySTMV.getPartPosition('m2', part)
            m3 = patchySTMV.getPartPosition('m3', part) 
            i1 = patchySTMV.getPartPosition('i1', part) 
            i2 = patchySTMV.getPartPosition('i2', part) 
            i3 = patchySTMV.getPartPosition('i3', part) 
            i4 = patchySTMV.getPartPosition('i4', part) 
            i5 = patchySTMV.getPartPosition('i5', part) 
            datafile.write('type_0' + ' ' + ' '.join(map(str, m1)) + '\n')
            datafile.write('type_1' + ' ' + ' '.join(map(str, m2)) + '\n')
            datafile.write('type_2' + ' ' + ' '.join(map(str, m3)) + '\n')
            datafile.write('type_3' + ' ' + ' '.join(map(str, i1)) + '\n')
            datafile.write('type_3' + ' ' + ' '.join(map(str, i2)) + '\n')
            datafile.write('type_4' + ' ' + ' '.join(map(str, i3)) + '\n')
            datafile.write('type_4' + ' ' + ' '.join(map(str, i4)) + '\n')
            datafile.write('type_5' + ' ' + ' '.join(map(str, i5)) + '\n')
    integrator.integrate(partlist)
    if i%5000 == 0:
        print("Percentage complete: ", 100*i/timesteps, "%", end="\r")
datafile.close()
# Generate TCL script to visualize with VMD
msmrdvis.generateTCL_patchyParticleSTMV(numparticles = numparticles, 
                                     outfname = "patchyParticleSTMV", 
                                     tclfname = "../../data/vmd/patchyParticleSTMV_2vmd.tcl")
print("Percentage complete: ", 100, " %")

To load the movie go to /data/vmd and run in a terminal "vmd -e patchyPacrticleSTMV_2vmd.tcl".

## In case VMD is not desired

In case VMD output is not desired, below we simply integrate the model and show the output directly.

In [ ]:
# Integrate particle list and print only positions of first and last particle in list
timesteps = 10
print('{:<10s}{:<15s}{:<40s}{:<40s}'.format("Iteration", "Time", "Position (first particle)", "Position (last particle)"))
for i in range(timesteps):
    print('{:<10d}{:<15f}{:<40s}{:<50s}'.format(i, integrator.clock, str(partlist[0].position), str(partlist[numparticles-1].position)))
    print(np.linalg.norm(partlist[0].position - partlist[1].position))
    integrator.integrate(partlist)

# Calculate potential and plot

Plots the potential for the case when bidning sites i3 and i4 are aligned in one axis

In [ ]:
# Calculate potential for aligned molecules interaction between i3 and i4
patchySTMValt = patchyParticleSTMV(100,20)

def patchyParticleSTMVevaluate(p1,p2,or1,or2):
    part1 = msmrd2.particle(1.0, 1.0, p1, or1)
    part2 = msmrd2.particle(1.0, 1.0, p2, or2)
    return patchySTMValt.evaluate(part1, part2)

dist = np.arange(0,10, 0.01)
pos1 = np.array([0, 0., 0.])
pos2 = np.array([[i, 0., 0.] for i in dist])
unitX = np.array([1,0,0])

# patches coordinates in rest position and orientation
I1 = np.array([1.694500527, 0.674187053, -0.694570924])
I2 = np.array([2.198599972, -0.207424062, -0.826036629])
I3 = np.array([-0.487943602, -0.924886815, -0.163915576])
I4 = np.array([-1.348049652, 0.841422688, -0.057270532])
I5 = np.array([-1.003689633, 1.739415955, -0.562472092])

# Orientation of particle 1 to have i1 in x-axis 
unitI1 = I1/np.linalg.norm(I1)
rotVector = np.cross(unitI1,unitX)
rotTheta = np.arcsin(np.linalg.norm(rotVector))
rotVector =  rotTheta * rotVector/np.linalg.norm(rotVector)
orientation1 = quats.angle2quat(rotVector)

# Orientation of particle 2 to have i2 in x-axis (looking towards the negtive side)
unitI2 = I2/np.linalg.norm(I2)
rotVector = np.cross(unitI2, unitX)
rotTheta = np.pi + np.arcsin(np.linalg.norm(rotVector))
rotVector =  rotTheta * rotVector/np.linalg.norm(rotVector)
orientation2 = quats.angle2quat(rotVector)

# Orientation of particle 3 to have i3 in x-axis
unitI3 = I3/np.linalg.norm(I3)
rotVector = np.cross(unitI3,unitX)
rotTheta = np.pi - np.arcsin(np.linalg.norm(rotVector))
rotVector =  rotTheta * rotVector/np.linalg.norm(rotVector)
orientation3 = quats.angle2quat(rotVector)

# Orientation of particle 4 to have i4 in x-axis (looking towards the negtive side)
unitI4 = I4/np.linalg.norm(I4)
rotVector = np.cross(unitI4, -1*unitX)
rotTheta = np.arcsin(np.linalg.norm(rotVector))
rotVector =  rotTheta * rotVector/np.linalg.norm(rotVector)
orientation4 = quats.angle2quat(rotVector)

# Orientation of particle 5 to have i5 in x-axis
unitI5 = I5/np.linalg.norm(I5)
rotVector = np.cross(unitI5, -1*unitX)
rotTheta = np.pi + np.arcsin(np.linalg.norm(rotVector))
rotVector =  rotTheta * rotVector/np.linalg.norm(rotVector)
orientation5 = quats.angle2quat(rotVector)

# Orientation of particle 6 to have i5 in x-axis (looking towards the negtive side)
rotVector = np.cross(unitI5, -1*unitX)
rotTheta = np.arcsin(np.linalg.norm(rotVector))
rotVector =  rotTheta * rotVector/np.linalg.norm(rotVector)
orientation6 = quats.angle2quat(rotVector)

potentialAlignedI1I2 = [patchyParticleSTMVevaluate(pos1,p2,orientation1,orientation2) for p2 in pos2]
potentialAlignedI3I4 = [patchyParticleSTMVevaluate(pos1,p2,orientation3,orientation4) for p2 in pos2]
potentialAlignedI5I5 = [patchyParticleSTMVevaluate(pos1,p2,orientation5,orientation6) for p2 in pos2]

In [ ]:
# Check orientation of binding sites are indeed in x-axis
orientation = np.array([1,0,0,0])
part1 = msmrd2.particle(1.0, 1.0, pos1, orientation1)
part2 = msmrd2.particle(1.0, 1.0, pos1, orientation2)
part3 = msmrd2.particle(1.0, 1.0, pos1, orientation3)
part4 = msmrd2.particle(1.0, 1.0, pos1, orientation4)
part5 = msmrd2.particle(1.0, 1.0, pos1, orientation5)
part6 = msmrd2.particle(1.0, 1.0, pos1, orientation6)

print(patchySTMValt.getPartPosition('i1', part1))
print(patchySTMValt.getPartPosition('i2', part2))
print(patchySTMValt.getPartPosition('i3', part3))
print(patchySTMValt.getPartPosition('i4', part4))
print(patchySTMValt.getPartPosition('i5', part5))
print(patchySTMValt.getPartPosition('i5', part6))

In [ ]:
# Plot potential
plt.figure(figsize=(7,5))
fsize = 22
plt.rcParams.update({'font.size': fsize})
plt.plot(dist, potentialAlignedI1I2, label = r'aligned i1-i2', lw =2)
plt.plot(dist, potentialAlignedI3I4, label = r'aligned i3-i4', lw =2)
plt.plot(dist, potentialAlignedI5I5, label = r'aligned i5-i5', lw =2)
plt.plot(dist, 0*dist, '--k', lw=0.5)
#plt.legend(fancybox=True)
plt.legend(fontsize=28, labelspacing=2, framealpha=1.0, edgecolor='white', fancybox=True, bbox_to_anchor=(1.3, 1.0))
plt.xlim([0,10])
plt.xlabel(r'Relative distance')
plt.ylabel(r'Potential')
plt.yticks([])
#plt.savefig('patchyParticles_potential.pdf', bbox_inches='tight')